In [ ]:
!pip install multion -q

In [ ]:
import multion

multion.login()

In [ ]:
# Now you can make API calls like this:
response = multion.new_session({"input": f"Weather Today","url": "https://www.google.com"})
print(response)
tabId = response['tabId']

In [ ]:
response = multion.update_session(tabId, {"input": f"Weather Today","url": "https://www.google.com"})
response

In [ ]:
# LANGCHAIN EXAMPLE
from langchain import LLMMathChain, OpenAI, SerpAPIWrapper, SQLDatabase, SQLDatabaseChain
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
import openai


import os

os.environ["LANGCHAIN_TRACING"] = "true"
os.environ['OPENAI_API_KEY'] = "<open_api_key>"


from langchain.tools import StructuredTool
TabId = None

def create_session(action: str, url: str) -> str:
    """Always the first step to run any activities that can be done using browser.
    The function parameters 'action' and 'url' both are compulsary.
    'action' is the action that you need to perform in the given url.if there is no 'action'set it as open.
    'url' is the base url of a site."""
    global TabId
    response = multion.new_session({"input": action,"url": url})
    response=response['response']['data']
    TabId = response['tabId']
    print(response)
    return  response['message']

def update_session(action:str,url:str) -> str:
    """Updates the existing browser session created using create_session with given action and url, used consequently to handle browser activitites after creating one session of browser.
    The function parameters 'action' and 'url' both are compulsary.
    'action' is the action that you need to perform in the given url.if there is no 'action'set it as open.
    'url' is the base url of a site."""
    global TabId
    response = multion.update_session(TabId, {"input": action,"url": url})
    response=response['response']['data']
    return  response['message']

tool1 = StructuredTool.from_function(create_session)
tool2= StructuredTool.from_function(update_session)


llm = OpenAI(temperature=0)

# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type.
agent_executor = initialize_agent(
    [tool1,tool2],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


In [ ]:
agent_executor.run(f"list and what their names are: https://www.notion.so/multion/Can-it-read-this-doc-9dbe753a121e473a8797828dda902b10?pvs=4")

In [ ]:
multion.refresh_token()